# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 10:28AM,238521,20,SSNA-2173645713,Sartorius Stedim North America,Released
1,Jun 1 2022 10:24AM,238520,10,NST-1038,"Amcyte Pharma, Inc.",Released
2,Jun 1 2022 10:24AM,238520,10,NST-1039,"Amcyte Pharma, Inc.",Released
3,Jun 1 2022 10:24AM,238520,10,NST-1040,"Amcyte Pharma, Inc.",Released
4,Jun 1 2022 10:24AM,238520,10,NST-1041,"Amcyte Pharma, Inc.",Released
5,Jun 1 2022 10:20AM,238519,10,Enova-10047,Emerginnova,Released
6,Jun 1 2022 10:20AM,238519,10,Enova-10048,Emerginnova,Released
7,Jun 1 2022 10:20AM,238519,10,Enova-10049,Emerginnova,Released
8,Jun 1 2022 10:03AM,238516,20,SSNA-2173638865,Sartorius Stedim North America,Released
9,Jun 1 2022 9:58AM,238443,15,PNC356140C,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
53,238515,Released,1
54,238516,Released,1
55,238519,Released,3
56,238520,Released,4
57,238521,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238515,NaN,NaN,NaN,1.0
238516,NaN,NaN,NaN,1.0
238519,NaN,NaN,NaN,3.0
238520,NaN,NaN,NaN,4.0
238521,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238318,0.0,0.0,0.0,20.0
238342,0.0,0.0,0.0,1.0
238343,0.0,0.0,0.0,1.0
238406,1.0,12.0,6.0,0.0
238409,0.0,0.0,0.0,41.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238318,0.0,0,0,20
238342,0.0,0,0,1
238343,0.0,0,0,1
238406,1.0,12,6,0
238409,0.0,0,0,41


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238318,0.0,0,0,20
1,238342,0.0,0,0,1
2,238343,0.0,0,0,1
3,238406,1.0,12,6,0
4,238409,0.0,0,0,41


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238318,0.0,,,20
1,238342,0.0,,,1
2,238343,0.0,,,1
3,238406,1.0,12,6,
4,238409,0.0,,,41


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc."
5,Jun 1 2022 10:20AM,238519,10,Emerginnova
8,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America
9,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc."
43,Jun 1 2022 9:52AM,238515,10,"Senseonics, Incorporated"
44,Jun 1 2022 9:45AM,238318,15,"Person & Covey, Inc."
64,Jun 1 2022 9:44AM,238512,10,"Senseonics, Incorporated"
65,Jun 1 2022 9:43AM,238492,15,"Mizner Bioscience, LLC"
67,Jun 1 2022 9:42AM,238490,15,"Laser Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America,0.0,,,1
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc.",0.0,,,4
2,Jun 1 2022 10:20AM,238519,10,Emerginnova,0.0,,,3
3,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America,0.0,,,1
4,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc.",0.0,,,34
5,Jun 1 2022 9:52AM,238515,10,"Senseonics, Incorporated",0.0,,,1
6,Jun 1 2022 9:45AM,238318,15,"Person & Covey, Inc.",0.0,,,20
7,Jun 1 2022 9:44AM,238512,10,"Senseonics, Incorporated",0.0,,,1
8,Jun 1 2022 9:43AM,238492,15,"Mizner Bioscience, LLC",0.0,,,2
9,Jun 1 2022 9:42AM,238490,15,"Laser Pharmaceuticals, LLC",0.0,,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America,1,,
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc.",4,,
2,Jun 1 2022 10:20AM,238519,10,Emerginnova,3,,
3,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America,1,,
4,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc.",34,,
5,Jun 1 2022 9:52AM,238515,10,"Senseonics, Incorporated",1,,
6,Jun 1 2022 9:45AM,238318,15,"Person & Covey, Inc.",20,,
7,Jun 1 2022 9:44AM,238512,10,"Senseonics, Incorporated",1,,
8,Jun 1 2022 9:43AM,238492,15,"Mizner Bioscience, LLC",2,,
9,Jun 1 2022 9:42AM,238490,15,"Laser Pharmaceuticals, LLC",22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America,1,,
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc.",4,,
2,Jun 1 2022 10:20AM,238519,10,Emerginnova,3,,
3,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America,1,,
4,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc.",34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc.",4.0,NaN,NaN
2,Jun 1 2022 10:20AM,238519,10,Emerginnova,3.0,NaN,NaN
3,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America,1.0,NaN,NaN
4,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc.",34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 10:28AM,238521,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 1 2022 10:24AM,238520,10,"Amcyte Pharma, Inc.",4.0,0.0,0.0
2,Jun 1 2022 10:20AM,238519,10,Emerginnova,3.0,0.0,0.0
3,Jun 1 2022 10:03AM,238516,20,Sartorius Stedim North America,1.0,0.0,0.0
4,Jun 1 2022 9:58AM,238443,15,"Person & Covey, Inc.",34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4292650,284.0,19.0,21.0
102,953885,3.0,1.0,0.0
12,476969,2.0,1.0,1.0
15,2145972,114.0,5.0,0.0
16,238486,0.0,1.0,0.0
18,715111,2.0,1.0,0.0
19,238495,17.0,11.0,0.0
20,1430937,38.0,49.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4292650,284.0,19.0,21.0
1,102,953885,3.0,1.0,0.0
2,12,476969,2.0,1.0,1.0
3,15,2145972,114.0,5.0,0.0
4,16,238486,0.0,1.0,0.0
5,18,715111,2.0,1.0,0.0
6,19,238495,17.0,11.0,0.0
7,20,1430937,38.0,49.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,284.0,19.0,21.0
1,102,3.0,1.0,0.0
2,12,2.0,1.0,1.0
3,15,114.0,5.0,0.0
4,16,0.0,1.0,0.0
5,18,2.0,1.0,0.0
6,19,17.0,11.0,0.0
7,20,38.0,49.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,284.0
1,102,Released,3.0
2,12,Released,2.0
3,15,Released,114.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,21.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
Executing,19.0,1.0,1.0,5.0,1.0,1.0,11.0,49.0
Released,284.0,3.0,2.0,114.0,0.0,2.0,17.0,38.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,21.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1,Executing,19.0,1.0,1.0,5.0,1.0,1.0,11.0,49.0
2,Released,284.0,3.0,2.0,114.0,0.0,2.0,17.0,38.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,21.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
1,Executing,19.0,1.0,1.0,5.0,1.0,1.0,11.0,49.0
2,Released,284.0,3.0,2.0,114.0,0.0,2.0,17.0,38.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()